# Programming PyTorch for Deep Learning
### Ian Pointer (O'Reilly)
### Notes and tests

This books assumes a working CUDA installation. Let's hope for the best...

## Chapter 1. Getting started with PyTorch
### Tensors
A tensor is both a container for numbers (like a vector or matrix) but also represents sets of rules defining transformations
between tensors that produce new tensors. Tensors have ranks that represent their dimensional space. Tensors with PyTorch
support fetching and changing elements using the standard Python indexing. 

In [2]:
import torch, torchvision, numpy, pandas

x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(x)
print(x[0][-1])  # fetch the last element of the first dimension
x[0][0] = 10  # change the value of the first element of the first dimension

# There are multiple functions that can create tensors, like torch.zeros(), torch.ones(), torch.rand()

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor(3)


#### Tensor operations
There are a lot.
For instance, we can find the maximum value in a tensor like this:

In [9]:
x = torch.rand(2, 2)
print(x)
print(x.max(), x.argmax(), x.max().item())

tensor([[0.9442, 0.7211],
        [0.1489, 0.1930]])
tensor(0.9442) tensor(0) 0.9441968202590942
tensor([0.9442, 0.7211])
tensor(0)


There are multiple types of tensors, for instance ```LongTensors``` or ```FloatTensors```. We can convert back and forth 
using the ```.to()``` method.

In [11]:
long_tensor = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]])
print(long_tensor.type())
float_tensor = long_tensor.to(dtype=torch.float32)
print(float_tensor.type())


torch.LongTensor
torch.FloatTensor
